#Obtenido los datos de yahoo - precio de cierre contra 200 dma
#Acumulado contra el último precio de cierre


In [2]:
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp
cf.set_config_file(offline=True)  # set the plotting mode to offline

In [3]:
cfg = cp.ConfigParser()
cfg.read('../refinitiv.cfg')
ek.set_app_key(cfg['eikon']['app_id'])

At the start of every calendar month, we sell all holdings and buy 50 random stocks. 
The weight of each stock is still completely random, but they all add up to 100% 
to make sure we’re fully invested. 
The only added rule here is that if the index itself is below its 200-day moving average,
we don’t buy. 
This means that if the index is below the average on the first of the month, 
we spend the entire month in cash.
Now we do another 500 random iterations to see what comes out.

In [4]:
FTSE = ek.get_data('0#.FTSE', fields=['TR.CommonName', 'TR.PriceClose',
                                      'TR.Volume', 'TR.TotalReturnYTD'])[0]
FTSE

,Instrument,Company Common Name,Price Close,Volume,YTD Total Return
0,STAN.L,Standard Chartered PLC,682.4,12963766,2.370237
1,CRDA.L,Croda International PLC,4792.0,291690,-5.108911
2,ANTO.L,Antofagasta PLC,1838.0,1336267,9.437333
3,BNZL.L,Bunzl plc,3066.0,829093,-3.887147
4,SGE.L,Sage Group PLC,1253.5,2551172,8.087451
...,...,...,...,...,...
95,EDV.L,Endeavour Mining plc,1368.0,1241781,-20.122631
96,OCDO.L,Ocado Group PLC,445.0,10734170,-41.32384
97,LSEG.L,London Stock Exchange Group PLC,9070.0,877055,-2.199698
98,TSCO.L,Tesco PLC,275.9,15338001,-5.025818


In [5]:
FTSC = ek.get_data('0#.FTSC', fields=['TR.CommonName', 'TR.PriceClose',
                                      'TR.Volume', 'TR.TotalReturnYTD'])[0]
FTSC

,Instrument,Company Common Name,Price Close,Volume,YTD Total Return
0,FSTA.L,Fuller Smith & Turner PLC,552.0,73431,-15.337423
1,IGC.L,India Capital Growth Fund Ltd,185.5,632302,7.225434
2,PINE.L,Pinewood Technologies Group PLC,37.0,5656009,14.551084
3,OIT.L,Odyssean Investment Trust PLC,151.5,101244,-4.113924
4,NCYF.L,CQS New City High Yield Fund Ltd,50.0,656150,0.774829
...,...,...,...,...,...
213,MNL.L,Manchester And London Investment Trust PLC,668.0,100221,22.120658
214,PORV.L,Porvair PLC,642.0,51448,4.220779
215,APTD.L,Aptitude Software Group PLC,320.0,4044,14.285714
216,PCA.L,Palace Capital PLC,223.0,31309,0.45045


In [6]:
import datetime as dt
today = dt.datetime.today()
f_inicio = today - dt.timedelta(days=600)
print(today, f_inicio)

2024-03-04 17:38:16.988244 2022-07-13 17:38:16.988244


In [7]:
df_ts = ek.get_timeseries(['.FTSE'], fields = ['CLOSE'],
                          start_date = f_inicio, 
                          end_date = today, 
                          interval="daily", 
                          corax = 'adjusted' )

df_ts

.FTSE,CLOSE
Date,
2022-07-14,7039.81
2022-07-15,7159.01
2022-07-18,7223.24
2022-07-19,7296.28
2022-07-20,7264.31
...,...
2024-02-27,7683.02
2024-02-28,7624.98
2024-02-29,7630.02


In [8]:
df_ts['SMA_200'] = df_ts['CLOSE'].rolling(200).mean()
filtered_df_ts = df_ts[~df_ts['SMA_200'].isna()]
filtered_df_ts

.FTSE,CLOSE,SMA_200
Date,,
2023-04-28,7870.57,7490.09390
2023-05-02,7773.03,7493.76000
2023-05-03,7788.37,7496.90680
2023-05-04,7702.64,7499.30380
2023-05-05,7778.38,7501.71430
...,...,...
2024-02-27,7683.02,7543.45715
2024-02-28,7624.98,7542.96590
2024-02-29,7630.02,7542.40450


In [9]:
# if the index itself is below its 200-day moving average, we don’t buy. 

filtered_df_ts['BUY'] = filtered_df_ts['SMA_200'] < filtered_df_ts['CLOSE']
filtered_df_ts

.FTSE,CLOSE,SMA_200,BUY
Date,,,
2023-04-28,7870.57,7490.09390,True
2023-05-02,7773.03,7493.76000,True
2023-05-03,7788.37,7496.90680,True
2023-05-04,7702.64,7499.30380,True
2023-05-05,7778.38,7501.71430,True
...,...,...,...
2024-02-27,7683.02,7543.45715,True
2024-02-28,7624.98,7542.96590,True
2024-02-29,7630.02,7542.40450,True


At the start of every calendar month, we sell all holdings and buy 50 random stocks. 
The weight of each stock is still completely random, but they all add up to 100% 


In [11]:
Inst_rndm = pd.DataFrame(FTSE['Instrument'].sample(n=50))
Inst_rndm['randNumCol'] = np.random.randint(1,10, size=len(Inst_rndm))
Inst_rndm['weight'] = Inst_rndm['randNumCol']/sum(Inst_rndm['randNumCol'] )
Inst_rndm

,Instrument,randNumCol,weight
38,INF.L,8,0.033898
62,IHG.L,6,0.025424
68,PSN.L,4,0.016949
53,JD.L,4,0.016949
74,SSE.L,8,0.033898
95,EDV.L,6,0.025424
85,GLEN.L,6,0.025424
89,UTG.L,9,0.038136
37,DGE.L,6,0.025424
99,LGEN.L,8,0.033898


In [12]:
sum(Inst_rndm['weight'])

0.9999999999999999

In [22]:
df_ts_rndm = ek.get_timeseries(['INF.L'], fields = ['CLOSE'],
                          start_date = dt.datetime(2024, 2, 1, 0, 0), 
                          end_date = dt.datetime(2024, 2, 29, 0, 0), 
                          interval="daily", 
                          corax = 'adjusted' )

Precios = df_ts_rndm = df_ts_rndm.rename(columns={'CLOSE': 'INF.L'})

In [23]:
Precios 

INF.L,INF.L
Date,
2024-02-01,775.4
2024-02-02,775.2
2024-02-05,785.6
2024-02-06,788.4
2024-02-07,792.0
2024-02-08,791.2
2024-02-09,799.0
2024-02-12,804.8
2024-02-13,792.2


In [24]:
for i in Inst_rndm['Instrument'][1:]:
    print(i)
    w = ek.get_timeseries([i], fields = ['CLOSE'],
                          start_date = dt.datetime(2024, 2, 1, 0, 0), 
                          end_date = dt.datetime(2024, 2, 29, 0, 0), 
                          interval="daily", 
                          corax = 'adjusted' )
    w1 = w.rename(columns={'CLOSE': i})
    Precios = Precios.join(w1, lsuffix="_left", rsuffix="_right")

IHG.L
PSN.L
JD.L
SSE.L
EDV.L
GLEN.L
UTG.L
DGE.L
LGEN.L
RIO.L
WEIR.L
DCC.L
SGRO.L
MRON.L
KGF.L
OCDO.L
SGE.L
SJP.L
SN.L
SBRY.L
AAL.L
SKG.L
ULVR.L
AV.L
BDEV.L
WTB.L
SDR.L
LSEG.L
LAND.L
BRBY.L
RTO.L
RMV.L
BEZG.L
ABF.L
RS1R.L
SHEL.L
PSON.L
SPX.L
WPP.L
REL.L
SMT.L
CTEC.L
TW.L
UU.L
EXPN.L
HLN.L
TSCO.L
FRES.L
HLMA.L


In [25]:
Precios

,INF.L,IHG.L,PSN.L,JD.L,SSE.L,EDV.L,GLEN.L,UTG.L,DGE.L,LGEN.L,...,REL.L,SMT.L,CTEC.L,TW.L,UU.L,EXPN.L,HLN.L,TSCO.L,FRES.L,HLMA.L
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-01,775.4,7474,1442.5,114.95,1670.5,1415,426.1,1012.0,2905.0,250.9,...,3289,764.0,238.6,146.55,1062.5,3322,325.75,282.9,522.0,2187
2024-02-02,775.2,7494,1446.5,113.0,1668.5,1357,417.6,1006.0,2936.5,253.5,...,3308,764.4,235.8,146.95,1057.0,3304,325.0,290.8,505.6,2167
2024-02-05,785.6,7516,1411.0,107.05,1679.0,1340,409.4,995.5,2984.0,249.3,...,3301,771.0,236.6,145.15,1049.5,3274,329.05,289.8,504.8,2151
2024-02-06,788.4,7570,1427.0,110.75,1669.5,1338,410.5,1005.0,2971.5,251.5,...,3287,782.0,238.2,147.05,1049.0,3297,329.5,289.9,511.0,2174
2024-02-07,792.0,7642,1439.0,107.35,1653.0,1307,402.4,1006.0,2940.0,244.1,...,3269,783.0,238.8,148.75,1044.5,3290,329.15,280.0,504.4,2182
2024-02-08,791.2,7634,1377.0,104.5,1575.5,1290,396.8,1005.0,2926.0,241.3,...,3301,789.0,234.6,145.25,1025.5,3338,323.0,280.4,493.2,2213
2024-02-09,799.0,7674,1400.5,104.0,1557.0,1286,385.05,993.5,2899.5,233.8,...,3350,794.0,239.0,146.9,1016.5,3358,319.45,280.5,476.7,2211
2024-02-12,804.8,7734,1419.0,105.65,1564.0,1323,394.25,1006.0,2890.5,235.5,...,3335,806.8,238.4,147.95,1032.0,3380,319.65,279.6,493.4,2204
2024-02-13,792.2,7554,1361.5,105.45,1547.0,1282,392.7,979.0,2860.5,230.8,...,3265,779.0,234.6,141.45,1026.5,3304,317.9,273.7,471.8,2151
